# 코로나 19 확진자 데이터 분석 프로젝트

#### 분석목표: 코로나 19 공공데이터와 다른 데이터를 합쳐 분석하여 의미있는 결과를 도출하자

데이터출처: 질병관리본부

In [128]:
import pandas as pd
import numpy as np

In [146]:
# 데이터 불러오기 (코로나바이러스 확진환자 발생현황)
covid_country = pd.read_excel("data/코로나바이러스감염증-19_확진환자_발생현황_221028.xlsx",sheet_name = "발생별(국내발생+해외유입), 사망")
covid_age = pd.read_excel("data/코로나바이러스감염증-19_확진환자_발생현황_221028.xlsx",sheet_name = "연령별(10세단위)")
covid_city = pd.read_excel("data/코로나바이러스감염증-19_확진환자_발생현황_221028.xlsx",sheet_name = "시도별(17개시도+검역)")
covid_sex = pd.read_excel("data/코로나바이러스감염증-19_확진환자_발생현황_221028.xlsx",sheet_name = "성별(남+여)")

In [147]:
# 각 열별로 인덱스 3부터 제대로된 값이 나옴

covid_country = covid_country[3:]
covid_age = covid_age[3:]
covid_city = covid_city[3:]
covid_sex = covid_sex[3:]

In [148]:
# 인덱스 3은 각 칼럼이름으로 넣어주기
# 누적값도 제거하고 인덱스 3에있던 컬럼이름도 제거
# reset_index해주어 원래있던 인덱스가 컬럼으로 들어감 제거해줘야함
covid_country = covid_country.rename(columns = covid_country.iloc[0])[2:].reset_index()

In [149]:
# index라는 컬럼 제거해주기 
covid_country = covid_country.drop(columns = "index")

In [150]:
# 나머지도 동일하게 전처리
covid_age = covid_age.rename(columns = covid_age.iloc[0])[2:].reset_index()
covid_city = covid_city.rename(columns = covid_city.iloc[0])[2:].reset_index()
covid_sex = covid_sex.rename(columns = covid_sex.iloc[0])[2:].reset_index()

In [151]:
covid_age = covid_age.drop(columns = "index")
covid_city = covid_city.drop(columns = "index")
covid_sex = covid_sex.drop(columns = "index")

In [152]:
# 결측치 확인
covid_country.isnull().sum()

일자         0
계(명)       0
국내발생(명)    0
해외유입(명)    0
사망(명)      0
dtype: int64

In [153]:
# 결측치 확인
covid_city.isnull().sum()

일자      0
계(명)    0
서울      0
부산      0
대구      0
인천      0
광주      0
대전      0
울산      0
세종      0
경기      0
강원      0
충북      0
충남      0
전북      0
전남      0
경북      0
경남      0
제주      0
검역      0
dtype: int64

In [154]:
# 결측치 확인
covid_sex.isnull().sum()

일자       0
계(명)     0
남성(명)    0
여성(명)    0
dtype: int64

In [155]:
# 결측치 확인
covid_age.isnull().sum()

일자        0
계(명)      0
0-9세      0
10-19세    0
20-29세    0
30-39세    0
40-49세    0
50-59세    0
60-69세    0
70-79세    0
80세이상     1
dtype: int64

In [156]:
covid_age[covid_age["80세이상"].isnull()]

,일자,계(명),0-9세,10-19세,20-29세,30-39세,40-49세,50-59세,60-69세,70-79세,80세이상
530,2021-07-03 00:00:00,794,48,99,211,138,143,100,46,9,NaN


In [157]:
# 각 값들의 0값이 "-"로 들어가있어 이것들을 결측치가 아니라 0으로 변환
# 결측치 확인결과 코로나10 발생 초기인 20년도 초에 코로나 환자가 없을경우에만 "-"으로 표현함 따라서 0으로 대체하는게 제일 합리적
covid_country = covid_country.replace("-",0)
covid_city = covid_city.replace("-",0)
covid_age = covid_age.replace("-",0)
covid_sex = covid_sex.replace("-",0)

In [164]:
covid_country

,일자,계(명),국내발생(명),해외유입(명),사망(명)
0,2020-01-20,1,0,1,0
1,2020-01-21,0,0,0,0
2,2020-01-22,0,0,0,0
3,2020-01-23,0,0,0,0
4,2020-01-24,1,0,1,0
...,...,...,...,...,...
1008,2022-10-24,14296,14242,54,10
1009,2022-10-25,43743,43672,71,17
1010,2022-10-26,40829,40748,81,26
1011,2022-10-27,34987,34932,55,26


In [165]:
covid_city

,일자,계(명),서울,부산,대구,인천,광주,대전,울산,세종,경기,강원,충북,충남,전북,전남,경북,경남,제주,검역
0,2020-01-20,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2020-01-21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2020-01-22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2020-01-23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2020-01-24,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,2022-10-24,14296,2434,493,555,937,344,450,293,132,4597,563,514,574,362,365,862,645,147,29
1009,2022-10-25,43743,10058,2310,1830,2640,843,1443,771,439,12885,1344,1369,1647,1045,1027,1892,2032,141,27
1010,2022-10-26,40829,7906,1854,1956,2534,972,1164,792,363,12020,1657,1318,1570,1205,963,2369,1878,280,28
1011,2022-10-27,34987,6848,1745,1556,2218,895,1076,726,342,10264,1300,1075,1302,1003,813,1970,1654,178,22


In [162]:
covid_age

,일자,계(명),0-9세,10-19세,20-29세,30-39세,40-49세,50-59세,60-69세,70-79세,80세이상
0,2020-01-20,1,0,0,0,1,0,0,0,0,0.0
1,2020-01-21,0,0,0,0,0,0,0,0,0,0.0
2,2020-01-22,0,0,0,0,0,0,0,0,0,0.0
3,2020-01-23,0,0,0,0,0,0,0,0,0,0.0
4,2020-01-24,1,0,0,0,0,0,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1008,2022-10-24,14296,1246,1865,1814,2171,2285,1869,1539,865,642.0
1009,2022-10-25,43743,2895,5934,5485,6470,6780,6196,5069,3230,1684.0
1010,2022-10-26,40829,2556,4918,5165,5801,5933,5719,5300,3190,2247.0
1011,2022-10-27,34987,2207,3969,4518,4863,5307,5046,4644,2692,1741.0


In [163]:
covid_sex

,일자,계(명),남성(명),여성(명)
0,2020-01-20,1,0,1
1,2020-01-21,0,0,0
2,2020-01-22,0,0,0
3,2020-01-23,0,0,0
4,2020-01-24,1,1,0
...,...,...,...,...
1008,2022-10-24,14296,6505,7791
1009,2022-10-25,43743,19291,24452
1010,2022-10-26,40829,17734,23095
1011,2022-10-27,34987,15134,19853


In [ ]:
# 현재까지 데이터 전처리 완료함 다음에 할꺼 월, 요일, 년도 별로 나누기, 데이터 프레임 결합?, 다른 데이터와 결합해서 비교할꺼 찾기
# 각 컬럼기준으로 결합할거기때문에 2020-01-20부터 2022-10-28까지 날짜 데이터가 각 행으로 있는것 찾으면됨
# 대안 1) 주식데이터(삼성전자, 코스피등등) 코로나 확진자수와 주식의 상관관계? 어떤영향?